#TODO: check those smiles that were not given in BindingDB

In [1]:
import json 
import pandas as pd 
import os 
import anndata as ad
import decoupler as dc
import numpy as np
import sys 
from local_utils import  map 

work_dir = '../output'
os.makedirs(f'{work_dir}/affinity', exist_ok=True)
# read smiles map 
with open(f'{work_dir}/affinity/map_smile_sm_name.json', 'r') as f:
    map_smile_sm_name = json.load(f)

smiles_de_train = list(map_smile_sm_name.keys())
sm_names_de_train = list(map_smile_sm_name.values())

/root/.local/lib/python3.10/site-packages/numba/np/ufunc/parallel.py:371: NumbaWarning: The TBB threading layer requires TBB version 2021 update 6 or later i.e., TBB_INTERFACE_VERSION >= 12060. Found TBB_INTERFACE_VERSION = 12050. The TBB threading layer is disabled.
  warnings.warn(problem)


## Map smile to sm_name

In [28]:
df_train = pd.read_parquet('../input/kaggle/input/open-problems-single-cell-perturbations/de_train.parquet')
map_smile_sm_name = {}
for smile, sm_name in zip(df_train.SMILES, df_train.sm_name):
    if smile not in map_smile_sm_name.keys():
        map_smile_sm_name[smile] = sm_name
with open(f'{work_dir}/affinity/map_smile_sm_name.json', 'w') as f:
    json.dump(map_smile_sm_name, f)

## Process binding data of BindingDB
We save it as sm_name:targets 

In [3]:
data = pd.read_csv(f'{work_dir}/affinity/database/BindingDB_All_202406.tsv', sep='\t', on_bad_lines='skip')

/tmp/ipykernel_290208/3427166776.py:1: DtypeWarning: Columns (8,9,10,11,12,13,15,17,20,21,22,23,27,28,32,33,35,36,45,46,47,48,50,51,52,53,54,55,57,58,59,60,62,63,64,65,66,67,69,70,71,72,74,75,76,77,78,79,81,82,83,86,87,88,89,90,91,93,94,95,98,99,100,101,102,103,105,106,107,110,111,112,113,114,115,117,118,119,122,123,124,125,126,127,129,130,131,134,135,136,137,138,139,146,147,148,149,150,151,158,159,160,161,162,163,170,172,173,174,175) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(f'{work_dir}/affinity/database/BindingDB_All_202406.tsv', sep='\t', on_bad_lines='skip')


In [30]:
import requests
def smiles_to_iupac(smiles_list):
    rep = "Names"
    results_map = {}
    CACTUS = "https://cactus.nci.nih.gov/chemical/structure/{0}/{1}"
    for smiles in smiles_list:
        url = CACTUS.format(smiles, rep)
        try:
            response = requests.get(url)
            response.raise_for_status()
            rr = response.text.split('\n')
            results_map[smiles] = rr
        except requests.exceptions.RequestException as e:
            # Handle HTTP request errors by adding NaN to the results map
            results_map[smiles] = float('nan')
    return results_map
smiles_all = data_subset.SMILES.unique()
rr = smiles_to_iupac(smiles_all)


In [33]:
list(rr.values())

[['N-(4-bromo-2-fluorophenyl)-6-methoxy-7-[(1-methylpiperidin-4-yl)methoxy]quinazolin-4-amine',
  'N-(4-bromo-2-fluoro-phenyl)-6-methoxy-7-[(1-methyl-4-piperidyl)methoxy]quinazolin-4-amine',
  'N-(4-bromo-2-fluorophenyl)-6-methoxy-7-[(1-methyl-4-piperidinyl)methoxy]-4-quinazolinamine',
  '(4-bromo-2-fluoro-phenyl)-[6-methoxy-7-[(1-methyl-4-piperidyl)methoxy]quinazolin-4-yl]amine',
  'N-(4-bromo-2-fluoro-phenyl)-6-methoxy-7-[(1-methylpiperidin-4-yl)methoxy]quinazolin-4-amine',
  '338992-00-0',
  '443913-73-3',
  'Vandetanib',
  'Vandetanib [INN]',
  'ZD 6474',
  'Zactima',
  'AZD-6474',
  'ZD-6474',
  'D06407',
  'Vandetanib (JAN)',
  'ZD6474',
  'ZD6',
  '4-Quninazolinamine, N-(4-bromo-2-fluorophenyl)-6-methoxy-7-((1-methyl-4-piperidinyl)methoxy)-',
  'N-(4-Bromo-2-fluorophenyl)-6-methoxy-7-((1-methyl-4-piperidinyl)methoxy)-4-quinazolinamine'],
 ['N-[5-[(5-tert-butyl-1,3-oxazol-2-yl)methylsulfanyl]-1,3-thiazol-2-yl]piperidine-4-carboxamide',
  'N-[5-[(5-tert-butyloxazol-2-yl)methylsulf

In [37]:
for smile, names in rr.items():
    
    if type(names)==list:
        continue
    print(names, smile)

nan O=C(Nc1nc2cccc(-c3ccc(CN4CCS(=O)(=O)CC4)cc3)n2n1)C1CC1
nan CC[C@H](Nc1ncnc2[nH]cnc12)c1nc2cccc(F)c2c(=O)n1-c1ccccc1
nan CC(C)C[C@H](NC(=O)CNC(=O)c1cc(Cl)ccc1Cl)B(O)O


In [12]:
# choose available compounds 
data_subset = data[data['Ligand SMILES'].isin(smiles_de_train)] 
# rename columns
data_subset = data_subset[['Ligand SMILES', 'UniProt (SwissProt) Entry Name of Target Chain']].reset_index(drop=True)
data_subset.columns = ['SMILES', 'prot']

# map smile to sm_name
data_subset['sm_name'] = data_subset.SMILES.map(map_smile_sm_name)

data_subset = data_subset.loc[~data_subset.prot.isna(), :]
all_prots = data_subset.prot.unique()
# map protname to genename 
prot_to_gene_map = map.protname_genename(all_prots)

# drop smile 
# del data_subset['SMILES']
# # group targets for compounds
# targets = data_subset.groupby('sm_name')['gene'].apply(list).to_dict()
# all_targets_bDB = np.unique(np.concatenate(list(targets.values())))
# # save
# with open(f'{work_dir}/affinity/curated/targets_bindingDB_all.json', 'w') as f:
#     json.dump(targets, f)

['VGFR1_HUMAN' 'VGFR2_HUMAN' 'FGFR1_HUMAN' 'EGFR_HUMAN' 'CDK2_HUMAN'
 'DPP4_HUMAN' 'DPP8_HUMAN' 'THA_CHICK' 'THB_RAT' 'MET_HUMAN' 'ABL1_HUMAN'
 'ALK_HUMAN' 'AURKA_HUMAN' 'AURKC_HUMAN' 'UFO_HUMAN' 'CD11B_HUMAN'
 'CD11A_HUMAN' 'CDK19_HUMAN' 'CDK7_HUMAN' 'CDK8_HUMAN' 'CDK9_HUMAN'
 'CLK1_HUMAN' 'CLK4_HUMAN' 'HMOX1_RAT' 'HMOX2_RAT' 'CSF1R_HUMAN'
 'DDR1_HUMAN' 'DDR2_HUMAN' 'EPHA3_HUMAN' 'EPHA7_HUMAN' 'EPHB4_HUMAN'
 'MK15_HUMAN' 'FLT3_HUMAN' 'VGFR3_HUMAN' 'FRK_HUMAN' 'KIT_HUMAN'
 'M4K3_HUMAN' 'M4K5_HUMAN' 'MERTK_HUMAN' 'MKNK1_HUMAN' 'MKNK2_HUMAN'
 'PGFRA_HUMAN' 'PGFRB_HUMAN' 'RET_HUMAN' 'SLK_HUMAN' 'STK33_HUMAN'
 'TIE1_HUMAN' 'TIE2_HUMAN' 'BLK_HUMAN' 'CTRO_HUMAN' 'FGR_HUMAN'
 'FYN_HUMAN' 'LCK_HUMAN' 'LYN_HUMAN' 'PTK6_HUMAN' 'TNI3K_HUMAN'
 'M3K20_HUMAN' 'ABL2_HUMAN' 'CDK3_HUMAN' 'CDK5_HUMAN' 'CLK2_HUMAN'
 'CSK_HUMAN' 'KC1D_HUMAN' 'KC1E_HUMAN' 'EPHA2_HUMAN' 'EPHA4_HUMAN'
 'EPHA5_HUMAN' 'EPHA8_HUMAN' 'EPHB1_HUMAN' 'EPHB2_HUMAN' 'FGFR2_HUMAN'
 'GAK_HUMAN' 'HCK_HUMAN' 'M4K4_HUMAN' 'MELK_HUMAN' 'MY

In [18]:
data_subset['gene'] = data_subset.prot.map(prot_to_gene_map)
data_subset = data_subset[~data_subset.gene.isna()].reset_index(drop=True)
data_subset

,SMILES,prot,sm_name,gene
0,COc1cc2c(Nc3ccc(Br)cc3F)ncnc2cc1OCC1CCN(C)CC1,VGFR1_HUMAN,Vandetanib,FLT1
1,COc1cc2c(Nc3ccc(Br)cc3F)ncnc2cc1OCC1CCN(C)CC1,VGFR2_HUMAN,Vandetanib,KDR
2,COc1cc2c(Nc3ccc(Br)cc3F)ncnc2cc1OCC1CCN(C)CC1,FGFR1_HUMAN,Vandetanib,FGFR1
3,COc1cc2c(Nc3ccc(Br)cc3F)ncnc2cc1OCC1CCN(C)CC1,EGFR_HUMAN,Vandetanib,EGFR
4,CC(C)(C)c1cnc(CSc2cnc(NC(=O)C3CCNCC3)s2)o1,CDK2_HUMAN,BMS-387032,CDK2
...,...,...,...,...
2420,Cn1c(=O)cc(N2CCC[C@@H](N)C2)n(Cc2ccccc2C#N)c1=O,DPP4_HUMAN,Alogliptin,DPP4
2421,Cn1c(=O)cc(N2CCC[C@@H](N)C2)n(Cc2ccccc2C#N)c1=O,DPP4_HUMAN,Alogliptin,DPP4
2422,Cn1c(=O)cc(N2CCC[C@@H](N)C2)n(Cc2ccccc2C#N)c1=O,DPP4_HUMAN,Alogliptin,DPP4
2423,Cn1c(=O)cc(N2CCC[C@@H](N)C2)n(Cc2ccccc2C#N)c1=O,DPP4_HUMAN,Alogliptin,DPP4


In [13]:
prot_to_gene_map

{'VGFR1_HUMAN': 'FLT1',
 'VGFR2_HUMAN': 'KDR',
 'FGFR1_HUMAN': 'FGFR1',
 'EGFR_HUMAN': 'EGFR',
 'CDK2_HUMAN': 'CDK2',
 'DPP4_HUMAN': 'DPP4',
 'DPP8_HUMAN': 'DPP8',
 'THA_CHICK': 'THRA',
 'THB_RAT': 'Thrb',
 'MET_HUMAN': 'MET',
 'ABL1_HUMAN': 'ABL1',
 'ALK_HUMAN': 'ALK',
 'AURKA_HUMAN': 'AURKA',
 'AURKC_HUMAN': 'AURKC',
 'UFO_HUMAN': 'AXL',
 'CD11B_HUMAN': 'CDK11B',
 'CD11A_HUMAN': 'CDK11A',
 'CDK19_HUMAN': 'CDK19',
 'CDK7_HUMAN': 'CDK7',
 'CDK8_HUMAN': 'CDK8',
 'CDK9_HUMAN': 'CDK9',
 'CLK1_HUMAN': 'CLK1',
 'CLK4_HUMAN': 'CLK4',
 'HMOX1_RAT': 'Hmox1',
 'HMOX2_RAT': 'Hmox2',
 'CSF1R_HUMAN': 'CSF1R',
 'DDR1_HUMAN': 'DDR1',
 'DDR2_HUMAN': 'DDR2',
 'EPHA3_HUMAN': 'EPHA3',
 'EPHA7_HUMAN': 'EPHA7',
 'EPHB4_HUMAN': 'EPHB4',
 'MK15_HUMAN': 'MAPK15',
 'FLT3_HUMAN': 'FLT3',
 'VGFR3_HUMAN': 'FLT4',
 'FRK_HUMAN': 'FRK',
 'KIT_HUMAN': 'KIT',
 'M4K3_HUMAN': 'MAP4K3',
 'M4K5_HUMAN': 'MAP4K5',
 'MERTK_HUMAN': 'MERTK',
 'MKNK1_HUMAN': 'MKNK1',
 'MKNK2_HUMAN': 'MKNK2',
 'PGFRA_HUMAN': 'PDGFRA',
 'PGFRB_H

Fetched: 500 / 559
Fetched: 559 / 559


## Broad data
Very low on targets (only 25)

In [2]:
with open(f'{work_dir}/affinity/curated/targets_bindingDB_all.json', 'r') as f:
    targets_bindingDB = json.load(f)

In [17]:
data_broad = pd.read_csv(f'{work_dir}/affinity/database/Repurposing_Hub_export.txt', sep='\t')
data_broad_subset = data_broad[data_broad.Name.isin(sm_names_de_train)]
data_broad_subset = data_broad_subset[['Name', 'Target']].reset_index(drop=True)
data_broad_subset.columns = ['sm_name', 'target']
data_broad_subset = data_broad_subset.groupby('sm_name')['target'].apply(list).to_dict()
# save
with open(f'{work_dir}/affinity/curated/targets_broad.json', 'w') as f:
    json.dump(data_broad_subset, f)

## Coverage on genes and TFs

In [37]:
with open(f'{work_dir}/affinity/curated/targets_bindingDB_all.json', 'r') as f:
    targets_bindingDB = json.load(f)
    all_targets_bDB = np.unique(np.concatenate(list(targets_bindingDB.values())))
    print('number of targets: ', len(all_targets_bDB))
with open(f'{work_dir}/affinity/curated/targets_broad.json', 'r') as f:
    targets_broad  = json.load(f)
    print('number of targets: ', len(np.unique(np.concatenate(list(targets_broad.values())))))
tfs_list = np.loadtxt(f'{work_dir}/utoronto_human_tfs_v_1.01.txt', dtype=str)
print('number of TFs: ', len(tfs_list))

number of targets:  503
number of targets:  25
number of TFs:  1639


In [52]:
import biomart
from pybiomart import Dataset
import requests

def verify_gene_symbols(gene_names, species='hsapiens'):
    # Connect to the Ensembl BioMart service
    dataset = Dataset(name=f'{species}_gene_ensembl', host='http://www.ensembl.org')
    
    # Query the dataset for the given gene names
    annot = dataset.query(attributes=['external_gene_name'])
    
    # Create a set of valid gene names
    valid_gene_names = set(annot['Gene name'].tolist())
    
    # Calculate valid and invalid gene names
    verified_gene_names = [gene for gene in gene_names if gene in valid_gene_names]
    invalid_gene_names = [gene for gene in gene_names if gene not in valid_gene_names]
    
    print(f'Valid genes: {len(verified_gene_names)} out of {len(gene_names)}')
    print(f'Invalid genes: {len(invalid_gene_names)}')
    
    return verified_gene_names, invalid_gene_names


def verify_gene_symbols_uniprot(gene_names):
    valid_gene_names = set()
    url = "https://www.uniprot.org/uploadlists/"

    for gene in gene_names:
        params = {
            'from': 'GENENAME',
            'to': 'GENENAME',
            'format': 'tab',
            'query': gene
        }
        response = requests.get(url, params=params)
        if response.status_code == 200 and gene in response.text:
            valid_gene_names.add(gene)

    verified_gene_names = [gene for gene in gene_names if gene in valid_gene_names]
    invalid_gene_names = [gene for gene in gene_names if gene not in valid_gene_names]
    
    print(f'Valid genes: {len(verified_gene_names)} out of {len(gene_names)}')
    print(f'Invalid genes: {len(invalid_gene_names)}')
    
    return verified_gene_names, invalid_gene_names

# verified_genes, invalid_genes = verify_gene_symbols_uniprot(all_targets_bDB)
verified_genes_b, invalid_genes_b = verify_gene_symbols(all_targets_bDB)


Valid genes: 288 out of 503
Invalid genes: 215


In [53]:
invalid_genes_b

['5HT2B',
 '5HT2C',
 'AA1R',
 'AA3R',
 'AAPK1',
 'AAPK2',
 'ABCBB',
 'ACK1',
 'ACM1',
 'ACM2',
 'ACM3',
 'ACM4',
 'ACM5',
 'ACV1B',
 'ACVL1',
 'ADA2A',
 'ADA2B',
 'ADA2C',
 'AL1A1',
 'ALBU',
 'APJ',
 'AVR2A',
 'AVR2B',
 'BMR1A',
 'BMR1B',
 'CAC1C',
 'CD11A',
 'CD11B',
 'CD5R1',
 'CHK1',
 'CHK2',
 'CP17A',
 'CP19A',
 'CP1A2',
 'CP2A6',
 'CP2B6',
 'CP2C8',
 'CP2C9',
 'CP2CJ',
 'CP2D6',
 'CP3A4',
 'CP51A',
 'CSK21',
 'CSK22',
 'CSKP',
 'CTRO',
 'DPP2',
 'DUSTY',
 'DYR1A',
 'DYR1B',
 'E2AK1',
 'E2AK2',
 'E2AK4',
 'ENPL',
 'ERG1',
 'F16P1',
 'FAAH1',
 'FAK1',
 'FAK2',
 'FDFT',
 'GCR',
 'HS90A',
 'HS90B',
 'IKKA',
 'IKKB',
 'IKKE',
 'KAPCA',
 'KAPCB',
 'KC1A',
 'KC1AL',
 'KC1D',
 'KC1E',
 'KC1G1',
 'KC1G2',
 'KC1G3',
 'KCC1A',
 'KCC1D',
 'KCC1G',
 'KCC2A',
 'KCC2B',
 'KCC2D',
 'KCC2G',
 'KCC4',
 'KGP1',
 'KGP2',
 'KKCC1',
 'KKCC2',
 'KPCD',
 'KPCD1',
 'KPCD2',
 'KPCD3',
 'KPCE',
 'KPCI',
 'KPCL',
 'KPCT',
 'KS6A1',
 'KS6A2',
 'KS6A3',
 'KS6A4',
 'KS6A5',
 'KS6A6',
 'KS6B1',
 'KSYK',
 'LMBL1'

In [50]:
# !pip install biomart

In [46]:
import pybiomart as pbm
def test_ensembl_host(gene_names, host, species):
    dataset = pbm.Dataset(name=species+'_gene_ensembl',  host=host)
    annot = dataset.query(attributes=['chromosome_name', 'transcription_start_site', 'strand', 'external_gene_name', 'transcript_biotype'])
    annot.columns = ['Chromosome', 'Start', 'Strand', 'Gene', 'Transcript_type']
    annot['Chromosome'] = annot['Chromosome'].astype('str')
    filter = annot['Chromosome'].str.contains('CHR|GL|JH|MT')
    annot = annot[~filter]
    annot.columns=['Chromosome', 'Start', 'Strand', 'Gene', 'Transcript_type']
    gene_names_release = set(annot['Gene'].tolist())
    ov=len([x for x in gene_names if x in gene_names_release])
    print('Genes recovered: ' + str(ov) + ' out of ' + str(len(gene_names)))
    return ov


test_ensembl_host(gene_names=all_targets_bDB, host='http://jan2020.archive.ensembl.org/', species='hsapiens')
    

Genes recovered: 285 out of 503


285

In [39]:
# do we have all the targets among target genes
np.intersect1d(all_targets_bDB, tfs_list).shape

(5,)

## Coverage on grn

In [19]:
# read nets
grn_model_names = ['collectRI', 'figr', 'celloracle', 'granie', 'ananse', 'scglue', 'scenicplus']

grn_models_dict = {}
for name in grn_model_names:
    grn_models_dict[name] = pd.read_csv(f'{work_dir}/benchmark/grn_models/{name}.csv', index_col=0)

In [26]:
# check the presence of target genes among tfs 
targets = targets_broad
all_targets = np.unique(np.concatenate(list(targets.values())))
for name, grn in grn_models_dict.items():
    print(name, np.intersect1d(all_targets, grn.source.unique())) 

collectRI ['AR']
figr []
celloracle ['AR']
granie []
ananse []
scglue ['AR']
scenicplus []


## Enrichment analysis

In [97]:
# read de_data
de_train = pd.read_parquet(f'{work_dir}/affinity/de_train.parquet')
de_train = de_train.reset_index()
sm_names = de_train.sm_name
sample = de_train.sm_name + '@'  + de_train.cell_type
de_train['sample'] = sample
de_train.set_index('sample', inplace=True)
de_train.drop(columns=['cell_type', 'sm_name'], inplace=True)
de_train.head(3)

gene,A1BG,A1BG-AS1,A2M,A2M-AS1,AAAS,AACS,AAGAB,AAK1,AAMDC,AAMP,...,ZUP1,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11B,ZYX,ZZEF1
sample,,,,,,,,,,,,,,,,,,,,,
5-(9-Isopropyl-8-methyl-2-morpholino-9H-purin-6-yl)pyrimidin-2-amine@B cells,0.826571,0.366722,0.605799,0.577766,-0.287037,0.293762,0.606627,0.367315,0.955483,-0.579767,...,-0.573711,0.549571,0.077775,0.411108,1.571559,0.803124,0.950774,-0.067148,0.171418,1.943000
5-(9-Isopropyl-8-methyl-2-morpholino-9H-purin-6-yl)pyrimidin-2-amine@Myeloid cells,1.144635,0.888033,0.294882,1.034652,-0.079176,0.521397,-2.105741,1.811037,0.213352,0.065167,...,-1.100933,0.367605,0.088639,-0.498617,0.899551,0.092172,-0.212104,0.403848,2.683611,0.591665
5-(9-Isopropyl-8-methyl-2-morpholino-9H-purin-6-yl)pyrimidin-2-amine@NK cells,-0.022487,-2.229070,0.078174,2.232399,0.386491,-0.312335,0.108862,-0.570145,0.384764,0.008786,...,0.111267,-2.372358,-0.228849,-1.313962,0.349919,1.207870,0.119070,0.869728,-0.756822,-0.155182


In [98]:
net = grn_models_dict['scenicplus']
tf_act, p_values = dc.run_ulm(de_train, net=net)

# we only want sm_name as sample
tf_act['sm_name'] = sm_names.values
tf_act.set_index('sm_name', inplace=True)

tf_act.shape

(537, 65)

## Evaluate presence of targets 

In [106]:
ep_count = {}
for sm_name, target_genes in targets.items():
    target_genes_n = len(target_genes)
    TFs = tf_act.columns
    print(target_genes_n, len(np.intersect1d(TFs, target_genes)))
    # aa
    # 
    # # get tf act for given compound
    # mask = tf_act.index.get_level_values('sm_name') == sm_name
    # tf_act_subset = tf_act[mask]
    # for index, activities in  tf_act_subset.iterrows():
    #     np.intersect()
    #     # print(np.sort(np.abs(activities)))
    #     aa


8 0
2 0
52 0
39 0
5 0
852 0
31 0
31 0
15 0
47 0
6 0
38 0
106 0
4 0
18 0
6 0
17 0
6 0
3 0
22 0
11 0
1011 0
